In [1]:
import mne

In [2]:
subjects_dir = 'D:\\Ekaterina_Voevodina\\MRI_FS\\'
subject = 'Sub2_memform'
src = mne.setup_source_space(
    subject, spacing='oct4', add_dist='patch',
    subjects_dir=subjects_dir)
print(src)

Setting up the source space with the following parameters:

SUBJECTS_DIR = D:\Ekaterina_Voevodina\MRI_FS
Subject      = Sub2_memform
Surface      = white
Octahedron subdivision grade 4

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading D:\Ekaterina_Voevodina\MRI_FS\Sub2_memform\surf\lh.white...
Mapping lh Sub2_memform -> oct (4) ...
    Triangle neighbors and vertex normals...
Loading geometry from D:\Ekaterina_Voevodina\MRI_FS\Sub2_memform\surf\lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 258/158411 selected to source space (oct = 4)

Loading D:\Ekaterina_Voevodina\MRI_FS\Sub2_memform\surf\rh.white...
Mapping rh Sub2_memform -> oct (4) ...
    Triangle neighbors and vertex normals...
Loading geometry from D:\Ekaterina_Voevodina\MRI_FS\Sub2_memform\surf\rh.sphere...
Setting up the triangulation for the decimated surface...
loaded rh.white 258/151372 selected to source space (oct = 4)

Calculating patch informat

In [3]:
print(src)

<SourceSpaces: [<surface (lh), n_vertices=158411, n_used=258>, <surface (rh), n_vertices=151372, n_used=258>] MRI (surface RAS) coords, subject 'Sub2_memform', ~27.3 MB>


In [4]:
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject=subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

Creating the BEM geometry...
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
inner skull CM is  -0.87 -19.58  35.45 mm
Surfaces passed the basic topology checks.
Complete.

Approximation method : Linear collocation

Homogeneous model surface loaded.
Computing the linear collocation solution...
    Matrix coefficients...
        inner skull (2562) -> inner skull (2562) ...
    Inverting the coefficient matrix...
Solution ready.
BEM geometry computations complete.


In [5]:
bem

<ConductorModel | BEM (1 layer)>